In [1]:
import bs4 as bs
import urllib.request
import pandas as pd
import re
#import pickle
#import time
#import progressbar



In [2]:
#http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=posglos&NrKadencji=8
#read list of gov sessions for both cadences

#list of sessions
#cadences = [7,8]
cadences = [7]

lines = []
for cadence in cadences:
    url = 'http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=posglos&NrKadencji='+str(cadence)
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source,'lxml')
    
    #list all lines - including day Id's
    for url in soup.find_all('a'):
        lines.append(url.get('href'))


len(lines)

259

In [3]:
#  'agent.xsp?symbol=listaglos&IdDnia=1389

In [4]:
#get all day Id's 
dayIds = []

for line in lines:
    if '&IdDnia=' in line:
        dayIds.append(line[-4:])

len(dayIds)

168

In [5]:
# ('1497', 8)

In [6]:
#http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=listaglos&IdDnia=1570
#get votes for each day
lines = []
for dayId in dayIds:
    url = 'http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=listaglos&IdDnia='+str(dayId)
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source,'lxml')
    
    for url in soup.find_all('a'):
        lines.append(url.get('href')+'&'+dayId)

len(lines)

28222

In [7]:
# 'agent.xsp?symbol=glosowania&NrKadencji=7&NrPosiedzenia=99&NrGlosowania=5'
#line = 'agent.xsp?symbol=glosowania&NrKadencji=7&NrPosiedzenia=99&NrGlosowania=5'
#print(cadence, " ", session, " ", vote)        

In [8]:
#get all sessionIds and voteIds 
sessions = []

for line in lines:
    if 'symbol=glosowania&NrKadencji' in line:
        split_line = re.split('=|&',line)
        
        for x,text in enumerate(split_line):
            if split_line[x] == 'NrKadencji':
                cadence = split_line[x+1]
                session = split_line[x+3]
                vote = split_line[x+5]
                dayId = split_line[x+6]
        
        sessions.append([cadence, session, vote, dayId])

len(sessions)

12584

In [9]:
sessions_df = pd.DataFrame(sessions)
sessions_df.columns = ['cadence','session','vote','dayId']
sessions_df = sessions_df.drop_duplicates()
sessions_df.count()

cadence    6292
session    6292
vote       6292
dayId      6292
dtype: int64

In [10]:
#sessions_df.tail()

In [11]:
#http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=glosowania&NrKadencji=8&NrPosiedzenia=33&NrGlosowania=1
#parse vote result

#bar = progressbar.Progressbar(max_value = len(sessions_df['cadence']))
#i=0

lines = []
for index, row in sessions_df.iterrows():
    url = 'http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=glosowania&NrKadencji='+str(row['cadence'])+'&NrPosiedzenia='+str(row['session'])+'&NrGlosowania='+str(row['vote'])
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source,'lxml')
    
    for url in soup.find_all('a'):
        lines.append(url.get('href')+"&"+str(row['cadence']))
    
    #bar.update(i)
    #i++
    
len(lines)


748169

In [12]:
#pickle lines
#pickle.dump(lines,open('vote_summaries.p','wb'))

In [13]:
#unpickle lines
#read_lines = pickle.load(open('vote_summaries.p','rb'))

In [14]:
#lines

In [15]:
#get all vote summaries - cadence, vote detail number, club
summary_results = []

for line in lines:
    if 'agent.xsp?symbol=klubglos&IdGlosowania=' in line:
        split_line = re.split('=|&',line)
        for x,text in enumerate(split_line):
            if split_line[x] == 'IdGlosowania':
                voteId = split_line[x+1]
                club = split_line[x+3]
                cadence = split_line[x+4]
        summary_results.append([cadence, voteId, club])    

len(summary_results)

46108

In [16]:
summary_results_df = pd.DataFrame(summary_results)
summary_results_df.columns = ['cadence','voteId','club']
summary_results_df = summary_results_df.drop_duplicates()
summary_results_df.count()

cadence    46108
voteId     46108
club       46108
dtype: int64

In [17]:
#http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=klubglos&IdGlosowania=46371&KodKlubu=PiS
#get individual votes

data = pd.DataFrame()
cols = ['number','name','vote']

for index, row in summary_results_df.iloc[0:15000].iterrows():
    url = 'http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=klubglos&IdGlosowania='+str(row['voteId'])+'&KodKlubu='+str(row['club'])
    #source = urllib.request.urlopen(url).read()
    #soup = bs.BeautifulSoup(source,'lxml')
    dfs = pd.read_html(url,header=0,encoding='utf_8')
    for df in dfs:
        df1 = df.iloc[:,0:3]
        df1.columns = cols
        df2 = df.iloc[:,3:]
        df2.columns = cols
        df3 = df1.append(df2)
        df3['cadence'] = row['cadence']
        df3['club'] = row['club']
        df3['voteId'] = row['voteId']
    data = data.append(df3)


In [18]:
data.shape

(916726, 6)

In [19]:
data2 = data.dropna()
data2.to_csv('voting_cadence_7_1_dropna.csv')

In [20]:
data2.shape

(910068, 6)

In [21]:
#http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=klubglos&IdGlosowania=46371&KodKlubu=PiS
#get individual votes

data = pd.DataFrame()
cols = ['number','name','vote']

for index, row in summary_results_df.iloc[15000:30000].iterrows():
    url = 'http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=klubglos&IdGlosowania='+str(row['voteId'])+'&KodKlubu='+str(row['club'])
    #source = urllib.request.urlopen(url).read()
    #soup = bs.BeautifulSoup(source,'lxml')
    dfs = pd.read_html(url,header=0,encoding='utf_8')
    for df in dfs:
        df1 = df.iloc[:,0:3]
        df1.columns = cols
        df2 = df.iloc[:,3:]
        df2.columns = cols
        df3 = df1.append(df2)
        df3['cadence'] = row['cadence']
        df3['club'] = row['club']
        df3['voteId'] = row['voteId']
    data = data.append(df3)


In [22]:
data.shape

(943612, 6)

In [23]:
data2 = data.dropna()
data2.to_csv('voting_cadence_7_2_dropna.csv')

In [24]:
data2.shape

(937167, 6)

In [25]:
#http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=klubglos&IdGlosowania=46371&KodKlubu=PiS
#get individual votes

data = pd.DataFrame()
cols = ['number','name','vote']

for index, row in summary_results_df.iloc[30000:].iterrows():
    url = 'http://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=klubglos&IdGlosowania='+str(row['voteId'])+'&KodKlubu='+str(row['club'])
    #source = urllib.request.urlopen(url).read()
    #soup = bs.BeautifulSoup(source,'lxml')
    dfs = pd.read_html(url,header=0,encoding='utf_8')
    for df in dfs:
        df1 = df.iloc[:,0:3]
        df1.columns = cols
        df2 = df.iloc[:,3:]
        df2.columns = cols
        df3 = df1.append(df2)
        df3['cadence'] = row['cadence']
        df3['club'] = row['club']
        df3['voteId'] = row['voteId']
    data = data.append(df3)


In [26]:
data.shape

(1051248, 6)

In [27]:
data2 = data.dropna()
data2.to_csv('voting_cadence_7_3_dropna.csv')

In [28]:
data2.shape

(1043244, 6)